In [1]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
from mlxtend.preprocessing import TransactionEncoder
import random
from mlxtend.frequent_patterns import apriori

In [2]:
#load data

client = boto3.client('s3')
obj = client.get_object(Bucket='manifolddata', Key='week1.csv')
df = pd.read_csv(BytesIO(obj['Body'].read()))

#df=pd.read_csv('week1processed.csv',  parse_dates=True)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df=df.iloc[:,[0,1,3,4,5,6,7,8]]
df.columns=['Date', 'Duration', 'Src_IP', 'Src_pt', 'Dst_IP', 'Dst_pt','Packets', 'Bytes']
#add an date column that is rounded to nearest hour, so we can use this as a timestep to see how frequently IP pairs occur in each timestep
df['Date']=pd.to_datetime(df['Date'], format="%Y-%m-%d %H:%M:%S.%f", errors = 'coerce')
df['date_hr']=pd.Series(df['Date']).dt.round("H")

In [4]:
#create a pair column, which is a touple of the src and dst IP, sorted. 
#It does not matter which call came first, we simply want to know which pair occurs most frequently.

df['pairs']=list(zip(df.Src_IP, df.Dst_IP))
df['pairs']=df['pairs'].apply(sorted)
df['pairs2']=tuple(df['pairs'])

In [8]:
df.head()

,Date,Duration,Src_IP,Src_pt,Dst_IP,Dst_pt,Packets,Bytes,date_hr,pairs,pairs2
0,2017-08-02 00:00:00.419,0.003,192.168.210.55,44870,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)"
1,2017-08-02 00:00:00.421,0.000,192.168.100.11,445,192.168.210.55,44870.0,1,108,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)"
2,2017-08-02 00:00:02.593,0.004,192.168.220.47,55101,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.220.47]","(192.168.100.11, 192.168.220.47)"
3,2017-08-02 00:00:02.859,0.000,10000_34,443,192.168.210.54,59628.0,1,100,2017-08-02,"[10000_34, 192.168.210.54]","(10000_34, 192.168.210.54)"
4,2017-08-02 00:00:02.594,0.000,192.168.100.11,445,192.168.220.47,55101.0,1,108,2017-08-02,"[192.168.100.11, 192.168.220.47]","(192.168.100.11, 192.168.220.47)"


In [6]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

In [7]:
dataset

[['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
 ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
 ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
 ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
 ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

In [8]:
#wants things in a 1 hot encoded pandas dataframe for the library models we want to run
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
data_coded = pd.DataFrame(te_ary, columns=te.columns_)
data_coded

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


In [38]:
df_pairs=df[['Date','pairs']]
df_pairs=pd.DataFrame(df_pairs)
df_pairs.columns

Index(['Date', 'pairs'], dtype='object')

Loop through in random chuncks, continue until no rules added

In [9]:
range(1,10)

range(1, 10)

In [25]:
#we only want a list of all the individual pairs at each timestamp. Think of this where each timestamp is a 'transaction' and we chose to buy which 2 items (IP addresses)
data_l=list(df['pairs'])

#due to memory issues, even though I am using an AWS instance with 32 GiB Memory, need to perform analysis on chucnks of data.
#We will keep going until no new rules are added
import random
data_l1=random.sample(data_l, 100000)

In [33]:
type(data_l1)

list

In [26]:
#wants things in a 1 hot encoded pandas dataframe for the library models we want to run
te = TransactionEncoder()
te_ary = te.fit(data_l1).transform(data_l1)
data1_coded = pd.DataFrame(te_ary, columns=te.columns_)
data1_coded.head()

,10000_34,10000_35,10000_40,10000_42,10000_66,10000_72,10000_74,10001_210,10001_213,10002_1,...,192.168.220.44,192.168.220.45,192.168.220.46,192.168.220.47,192.168.220.48,192.168.220.49,192.168.220.50,192.168.220.51,255.255.255.255,DNS
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [29]:
#wants things in a 1 hot encoded pandas dataframe for the library models we want to run. use Sprase data
te = TransactionEncoder()
te_ary = te.fit(data_l).transform(data_l, sparse=True)
data_coded = pd.SparseDataFrame(te_ary, columns=te.columns_, default_fill_value=False)
data_coded.head()


,0.0.0.0,10000_163,10000_34,10000_35,10000_40,10000_42,10000_66,10000_67,10000_72,10000_74,...,192.168.220.44,192.168.220.45,192.168.220.46,192.168.220.47,192.168.220.48,192.168.220.49,192.168.220.50,192.168.220.51,255.255.255.255,DNS
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False


In [57]:
data_subset=data_coded.sample(n=150000)
data_subset2=data_coded.sample(n=150000)

In [70]:
ap1=apriori(data_subset, min_support=0.005)
ap2=apriori(data_subset2, min_support=0.005)

In [105]:
ls_test[1] in ls_test

True

In [103]:
ap1_wet=ap1['itemsets']
ls_test=list(ap1_wet)

In [104]:
ls_test[1]

frozenset({94})

In [102]:
ap1_wet

0             (83)
1             (94)
2            (107)
3            (234)
4            (260)
5            (290)
6            (321)
7            (482)
8            (552)
9            (555)
10           (562)
11           (915)
12           (952)
13          (1553)
14          (1617)
15          (1649)
16          (1751)
17          (1753)
18          (1794)
19          (8576)
20          (9271)
21          (9272)
22          (9273)
23          (9274)
24          (9275)
25          (9277)
26          (9279)
27          (9280)
28          (9281)
29          (9282)
30          (9283)
31          (9284)
32          (9335)
33          (9336)
34          (9338)
35          (9339)
36          (9340)
37          (9341)
38          (9342)
39          (9343)
40          (9344)
41          (9346)
42      (9336, 83)
43      (9282, 94)
44    (9282, 9346)
45    (9346, 9335)
46    (9336, 9346)
47    (9338, 9346)
48    (9346, 9340)
49    (9346, 9341)
50    (9346, 9342)
51    (9346, 9343)
52    (9344,

In [90]:
mySet = frozenset(['Apple', 'Banana', 'Mango', 'Orange', 'Apple'])
myset2= frozenset(['Apple', 'Banana', 'meat'])



In [79]:
for i in myset2:
    if i in mySet:
        print(i)

Apple
Banana


In [86]:
ap1['itemsets'][1] in ap1['itemsets']

False

In [83]:
#check if sets are in either apriori selected rules

for i in len(ap1['itemsets']):
    if ap1['itemsets'][i] not in ap2['itemsets']:
        print(i)

TypeError: 'int' object is not iterable

In [42]:
masterdf=pd.DataFrame(columns=['support', 'itemsets'])

In [43]:
masterdf

,support,itemsets


In [37]:
#start with 10 loops
masterdf=DataFrame()
for i in range(0,10):
    data_subset=data_coded.sample(n=100000)
    ap=apriori(data_subset, min_support=0.05, use_colnames=True)
    if ap


IndexError: list assignment index out of range

now we will set some thresholds for creating rules and go through our first chunk of data


min_support : float (default: 0.5)
A float between 0 and 1 for minumum support of the itemsets returned. The support is computed as the fraction transactions_where_item(s)_occur / total_transactions.

We don't want to bother making rules for infrequent items, so we will set the support = 0.5 meaning this IP address is present in 50% of the requests


In [13]:
from mlxtend.frequent_patterns import apriori

ap1=apriori(data1_coded, min_support=0.05)


In [39]:
ap1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
support     6 non-null float64
itemsets    6 non-null object
dtypes: float64(1), object(1)
memory usage: 176.0+ bytes


In [21]:
ap1.itemsets[3]==({4855})

True

In [22]:
if ({4855}) in ap1.itemsets:
    print (ap1.itemsets)

TypeError: unhashable type: 'set'

In [24]:
df_pairs=df[['Date','pairs']]
df_pairs=pd.DataFrame(df_pairs)
df_pairs.columns

Index(['Date', 'pairs'], dtype='object')

In [28]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df_pairs2 = df_pairs.join(pd.DataFrame(mlb.fit_transform(df_pairs.pop('pairs')),
                          columns=mlb.classes_,
                          index=df_pairs.index))

MemoryError: 